In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib

### Load and describe data

In [4]:
target = 'System_Load'
raw = pd.read_csv(str(pathlib.Path("../data/interim/integrated.csv")), 
                  parse_dates=True, infer_datetime_format=True, index_col=0)
raw.pop('Date')
raw_stats = raw.describe(include='all')
total = raw_stats.loc['count', target]
missing_obs = raw_stats.loc['count'].apply(lambda x : round((total-x) / total * 100,2))
raw_stats.loc['missing obs (%)'] = missing_obs
raw_stats

,Hr_End,DA_Demand,RT_Demand,DA_LMP,DA_EC,DA_CC,DA_MLC,RT_LMP,RT_EC,RT_CC,...,Dry_Bulb,Dew_Point,System_Load,Reg_Service_Price,Reg_Capacity_Price,Min_5min_RSP,Max_5min_RSP,Min_5min_RCP,Max_5min_RCP,Hr_Start
count,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,...,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,8760.000000,8760.000000,8760.000000,8760.000000,25560.000000
mean,12.500000,13479.985440,13803.370914,35.414005,35.381176,-0.084724,0.117555,35.283515,35.117882,0.062338,...,51.603756,39.981690,14021.743662,0.340485,28.230619,0.126790,0.387572,21.026613,39.597312,11.500000
std,6.922322,2658.552075,2744.198544,25.260450,25.079650,1.095290,0.290586,39.468799,39.201921,1.748800,...,18.603842,19.485454,2774.836962,0.866514,39.176185,0.534386,1.127495,29.422158,65.538971,6.922322
min,1.000000,7345.700000,8201.860000,0.000000,0.000000,-65.110000,-0.860000,-156.040000,-155.870000,-68.000000,...,-11.000000,-25.000000,8398.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.750000,11560.975000,11845.370000,21.840000,21.790000,0.000000,-0.010000,19.870000,19.750000,0.000000,...,37.000000,26.000000,12049.000000,0.050000,11.390000,0.010000,0.050000,8.860000,13.290000,5.750000
50%,12.500000,13282.650000,13646.805000,28.500000,28.500000,0.000000,0.060000,26.670000,26.580000,0.000000,...,52.000000,42.000000,13853.500000,0.170000,17.440000,0.050000,0.090000,12.400000,22.850000,11.500000
75%,18.250000,14961.325000,15338.855000,39.770000,39.800000,0.010000,0.160000,39.840000,39.912500,0.010000,...,67.000000,56.000000,15570.250000,0.250000,31.570000,0.090000,0.250000,21.655000,43.970000,17.250000
max,24.000000,24754.100000,25591.520000,320.050000,314.880000,5.850000,4.820000,2454.570000,2446.710000,32.550000,...,94.000000,74.000000,25944.000000,10.000000,2331.550000,10.000000,10.000000,1087.200000,2580.420000,23.000000
missing obs (%),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,65.730000,65.730000,65.730000,65.730000,0.000000


### Select columns of interest

In [5]:
# Discard RCP and RSP due to sparseness of data.
select = raw.drop(columns=['Hr_End', 'Max_5min_RCP', 'Max_5min_RSP', 'Min_5min_RCP', 'Min_5min_RSP'])

In [6]:
stats = select.describe()
stats.loc['IQR'] = (stats.loc['75%'] - stats.loc['25%'])
stats.loc['low'] = stats.loc['25%'] - (1.5 * stats.loc['IQR'])
stats.loc['high'] = stats.loc['75%'] + (1.5 * stats.loc['IQR'])
stats.loc['missing'] = select.isna().sum()
stats

,DA_Demand,RT_Demand,DA_LMP,DA_EC,DA_CC,DA_MLC,RT_LMP,RT_EC,RT_CC,RT_MLC,Dry_Bulb,Dew_Point,System_Load,Reg_Service_Price,Reg_Capacity_Price,Hr_Start
count,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000,25560.000000
mean,13479.985440,13803.370914,35.414005,35.381176,-0.084724,0.117555,35.283515,35.117882,0.062338,0.103294,51.603756,39.981690,14021.743662,0.340485,28.230619,11.500000
std,2658.552075,2744.198544,25.260450,25.079650,1.095290,0.290586,39.468799,39.201921,1.748800,0.291958,18.603842,19.485454,2774.836962,0.866514,39.176185,6.922322
min,7345.700000,8201.860000,0.000000,0.000000,-65.110000,-0.860000,-156.040000,-155.870000,-68.000000,-1.350000,-11.000000,-25.000000,8398.000000,0.000000,0.000000,0.000000
25%,11560.975000,11845.370000,21.840000,21.790000,0.000000,-0.010000,19.870000,19.750000,0.000000,-0.020000,37.000000,26.000000,12049.000000,0.050000,11.390000,5.750000
50%,13282.650000,13646.805000,28.500000,28.500000,0.000000,0.060000,26.670000,26.580000,0.000000,0.050000,52.000000,42.000000,13853.500000,0.170000,17.440000,11.500000
75%,14961.325000,15338.855000,39.770000,39.800000,0.010000,0.160000,39.840000,39.912500,0.010000,0.140000,67.000000,56.000000,15570.250000,0.250000,31.570000,17.250000
max,24754.100000,25591.520000,320.050000,314.880000,5.850000,4.820000,2454.570000,2446.710000,32.550000,7.860000,94.000000,74.000000,25944.000000,10.000000,2331.550000,23.000000
IQR,3400.350000,3493.485000,17.930000,18.010000,0.010000,0.170000,19.970000,20.162500,0.010000,0.160000,30.000000,30.000000,3521.250000,0.200000,20.180000,11.500000
low,6460.450000,6605.142500,-5.055000,-5.225000,-0.015000,-0.265000,-10.085000,-10.493750,-0.015000,-0.260000,-8.000000,-19.000000,6767.125000,-0.250000,-18.880000,-11.500000


In [ ]:
# Observe: 0 missing entries for all data columns. No need for data imputation.